In [ ]:
%pip install bs4 selenium webdriver_manager python-dotenv haversine pandas

In [29]:
from bs4 import BeautifulSoup
from navigator import KakaoRouteFinder
import pandas as pd
import navigator
import coordinate
import os

os.environ['WDM_LOG_LEVEL'] = '0'

org, des = ('반포한강공원', '여의도한강공원')

org_coord = coordinate.get_coordinate(org)
des_coord = coordinate.get_coordinate(des)

(org_x, org_y) = coordinate.transform(org_coord[::-1], 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = coordinate.transform(des_coord[::-1], 'WGS84', 'WCONGNAMUL')

print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

routes = {}
routes['transit'] = KakaoRouteFinder().find_route_by_keyword('transit', org, des, time_delta=0.5, init_time=2.0)
navigator.extract_route('transit', routes['transit'])

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=498813,1113952,485664,1118587&rt1=반포한강공원&rt2=여의도한강공원


[{'time': '39분', 'info': '도보21분 | 환승2회 | 요금 1,250원 | 10.0km'},
 {'time': '45분', 'info': '도보27분 | 환승1회 | 요금 1,250원 | 9.8km'},
 {'time': '48분', 'info': '도보21분 | 환승1회 | 요금 1,250원 | 9.9km'},
 {'time': '52분', 'info': '도보16분 | 환승1회 | 요금 1,300원 | 12.0km'},
 {'time': '48분', 'info': '도보24분 | 환승1회 | 요금 1,200원 | 9.9km'},
 {'time': '56분', 'info': '도보17분 | 환승1회 | 요금 1,300원 | 11.9km'},
 {'time': '1시간 4분', 'info': '도보24분 | 환승없음 | 요금 1,200원 | 12.3km'},
 {'time': '46분', 'info': '도보23분 | 환승2회 | 요금 1,250원 | 9.7km'},
 {'time': '59분', 'info': '도보39분 | 환승없음 | 요금 1,200원 | 9.8km'},
 {'time': '54분', 'info': '도보33분 | 환승2회 | 요금 1,250원 | 11.4km'}]

In [36]:
from tqdm import tqdm

soup = BeautifulSoup(routes['transit'], 'html.parser')
S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 환승", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)

seoul_bike = {}

for lat, lng in [org_coord, des_coord]:
    try:
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        pass

for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = coordinate.df_bus[coordinate.df_bus['정류소명'] == s]
    subway = coordinate.df_subway[coordinate.df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass

rows = []
for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    rows.append([name, coord, org_dist-des_dist, congnamul])

df = pd.DataFrame(rows, columns=["name", "coord", "gap_dist", "congnamul"])
df.sort_values(by="gap_dist", ascending=False)

Retrieve Seoul Bike Stations: 100%|██████████| 55/55 [00:04<00:00, 11.45it/s]


,name,coord,gap_dist,congnamul
13,여의나루역 1번출구 앞,"(37.52715683, 126.9319)",5.565773,"(484951.0, 1118801.0)"
28,국민일보 앞,"(37.52881622, 126.92453)",5.564689,"(483323.0, 1119263.0)"
31,초원아파트 앞,"(37.53105545, 126.9242096)",5.555162,"(483252.0, 1119884.0)"
6,IFC몰,"(37.52603149, 126.9255066)",5.511563,"(483538.0, 1118490.0)"
10,유진투자증권빌딩 앞,"(37.52461243, 126.9278336)",5.447081,"(484052.0, 1118096.0)"
0,여의도고교 앞,"(37.52483749, 126.934906)",5.356213,"(485615.0, 1118157.0)"
20,KT앞,"(37.52190781, 126.9189529)",5.300118,"(482089.0, 1117347.0)"
39,여의도역 1번출구 옆,"(37.5213623, 126.9234619)",5.238521,"(483085.0, 1117195.0)"
21,신길역 2번출구,"(37.51756287, 126.9169846)",5.026785,"(481653.0, 1116142.0)"
2,미성아파트 A동 앞,"(37.5193634, 126.9260483)",5.024814,"(483656.0, 1116639.0)"


In [25]:
finder = KakaoRouteFinder()
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
bike_routes = {}
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in tqdm(range(i+1, len(bikes))):
        end = bikes[j]
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
        route = finder.find_route_by_congnamul('bike', srt_coord, end_coord, rt1=str(i), rt2=str(j), verbose=False, init=False, time_delta=0.2)
        route_id = f'{i}-{j}'
        bike_routes[route_id] = navigator.extract_route('bike', route)
del finder

100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
Finding Bike Routes: : 39it [03:13,  4.95s/it]


In [26]:
finder = KakaoRouteFinder()

org_routes = []
des_routes = []

for i, bike in tqdm(enumerate(bikes), desc="Finding Transit Routes"):
    (_, (name, _, _, _, _, coord)) = bike
    try:
        route = finder.find_route_by_congnamul('transit', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=True, time_delta=0.5, init_time=2)
        org_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=False, time_delta=0.5)
        org_routes.append(navigator.extract_route('walk', route))
        
    try:
        route = finder.find_route_by_congnamul('transit', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=True, time_delta=0.5, init_time=2)
        des_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=False, time_delta=0.5)
        des_routes.append(navigator.extract_route('walk', route))

del finder

Finding Transit Routes: 39it [05:09,  7.93s/it]


In [27]:
import pickle

filename = f'{org}-{des}.pkl'

with open(filename, 'wb') as f:
    data = {'org_routes': org_routes, "des_routes": des_routes, "bikes": bikes, "bike_routes": bike_routes }
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)